# **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [15]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [16]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [17]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    data = np.array(data) # transfer data into numpy array
    np.random.shuffle(data) # shuffle data
    split_index = int(len(data) * split_ratio) # calculate data length to split
    training_data = data[:split_index] # Put data[0] to data[split_index-1] into training_data
    validation_data = data[split_index:] # Put data[split_index] to the last element to validation_data

    # TODO

    return training_data, validation_data

#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [18]:
def fill_value(row):
    if pd.isna(row[1]):  # 检查是否为 NaN
        return 0.0 if row[2] < 168 else 1.0
    return row[1]

def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    
    preprocessData = data[:,5:6] * data[:,6:7] / data[:,4:5] * data[:,3:4]
    #print(f"pre1 : {preprocessData.shape}")
    preprocessData = np.column_stack((preprocessData,data))
    #print(f"pre2 : {preprocessData.shape}")

    # Change to pandas.DataFrame if it is numpy.array
    if isinstance(preprocessData, np.ndarray):
        preprocessData = pd.DataFrame(preprocessData)
    else:
        preprocessData = preprocessData.copy()

    # if preprocessData.shape[1] > 2:
    #     #print(f"columns : {preprocessData.columns.tolist()}")
    #     preprocessData[1] = preprocessData.apply(fill_value,axis=1)

    # Replace missing values with the median of their respective columns
    preprocessData.fillna(preprocessData.median(), inplace=True)

    # Handle outliers
    for column in preprocessData.columns:
        # Calculate the mean and standard deviation
        mean = preprocessData[column].mean()
        std = preprocessData[column].std()

        # Calculate the IQR
        Q1 = preprocessData[column].quantile(0.25)
        Q3 = preprocessData[column].quantile(0.75)
        IQR = Q3-Q1

        # Define lower and upper bound to check outliers
        Lower_bound = Q1-1.5*IQR
        Upper_bound = Q3+1.5*IQR

        # is_outlier = (preprocessData[column]<Lower_bound | preprocessData[column > Upper_bound])

        # # Define the threshold for outliers
        # threshold = 3 * std
        
        # # Identify outliers
        # is_outlier = (preprocessData[column] - mean).abs() > threshold
        # Replace outliers with the mean
        # preprocessData.loc[is_outlier, column] = IQR

        # Keep not outliers
        preprocessData = preprocessData[(preprocessData[column] >= Lower_bound) & (preprocessData[column] <= Upper_bound)]
        # Replace outliers with IQR
        # preprocessData[column] = preprocessData[column].clip(Lower_bound, Upper_bound)
        # preprocessData[column] = np.where(preprocessData[column] < Lower_bound, mean,
        #                          np.where(preprocessData[column] > Upper_bound, mean, preprocessData[column]))

    # Remove duplicated values
    # print(f"Duplicated : {preprocessData.duplicated().sum()}")
    #print(f"pre3 : {preprocessData.columns}")
    preprocessData = preprocessData.drop_duplicates()
    #data_standardized = (preprocessData - preprocessData.mean())/preprocessData.std()
    #print(f"pre4 : {preprocessData.columns}")
    return preprocessData.values


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [19]:
def Regression(Training,Validation,degree=2 ,Rseed = 1999,lambda_w=-18,num_iteration = 1000000,learning_rate = 0.000001):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """
    np.random.seed(Rseed)
    # Get X and y from dataset
    X = Training[:, : -1]
    y = Training[:, -1]
    Xv = Validation[:, :-1]
    yv = Validation[:,-1]
    # TODO: Decide on the degree of the polynomial
    # For example, quadratic regression

    X_ = np.ones(X.shape[0])
    X_ = np.column_stack((X_,X))
    Xv_ = np.ones(Xv.shape[0])
    Xv_ = np.column_stack((Xv_,Xv))
    #print(f"X_ : {X_.shape}")

    # # Add polynomial features to X
    # X_poly = np.ones((X.shape[0], X.shape[1]))  # Add intercept term (column of ones)
    # Xv_poly = np.ones((Xv.shape[0], Xv.shape[1]))
    # print(f"X_poly0:{X_poly.shape}")
    # for d in range(1, degree + 1):
    #     #print(f"d={d}")
    #     X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix
    #     Xv_poly = np.hstack((Xv_poly, Xv ** d))
    #     #print(f"X_poly1:{X_poly.shape}")
    # print(f"X:{X.shape}")
    # print(f"X_poly2:{X_poly.shape}")
    # Initialize coefficients (weights) to zero
    num_dimensions = X_.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.random.rand(num_dimensions)


    # TODO: Set hyperparameters
    #noise = 0
    prev_loss = float('inf')
    prev_gradient_length = float('inf')
    # print(f"X.shape={X.shape},num_dimensions={num_dimensions}")
    # Gradient Descent
    m = len(y)  # Number of data points
    History_loss = []
    History_Vloss = []
    History_gradient = []
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        
        y_pred = X_.dot(w) 
        y_predv = Xv_.dot(w)
        error = y - y_pred
        errorv = yv - y_predv

        # TODO: Compute gradient
        #gradient_w0 = -2/m * np.sum(error)
        gradient = -2/m * X_.T.dot(error) + lambda_w * w# gradient of w1
        loss = np.mean(error ** 2)
        lossv = np.mean(errorv ** 2)

        History_Vloss.append(lossv)
        History_loss.append(loss)
        History_gradient.append(gradient)

        #print(f"y_pred:{y_pred.shape} , X_poly:{X_poly.shape} , w:{w.shape} , gradient:{gradient.shape}")
        #print(f"y_pred = {y_pred}\n Error = {error}\n gradient = {gradient}")
        # TODO: Update the weights
        gradient_length = np.linalg.norm(gradient)
        #print(f"loss:{loss} , gradientL:{gradient_length} , error:{error[0]} , w:{w}")
        if lossv > prev_loss or abs(lossv - prev_loss) < 0.0000001:
            final_loss=lossv
            #print(f"Stop at {iteration}, loss:{lossv}, prev_loss:{prev_loss}")
            break
        prev_loss = lossv
        prev_gradient_length = gradient_length
        w = w - learning_rate * gradient
        #w0 = w0 - learning_rate * gradient_w0


        # # TODO: Optionally, print the cost every 100 iterations
        if iteration % 5000 == 0:
            cost = np.mean(error ** 2) / 2
            print(f"Iteration {iteration}, Cost: {cost}, w:{w}, loss:{lossv}, gradient:{gradient_length}")
    # print(f"w:{w}, loss:{prev_loss}, noise:{w[:7]}, gradient:{prev_gradient_length}")
    return w[1:],History_loss,History_gradient,History_Vloss,lossv#,w0


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [20]:
def MakePrediction(w, test_dataset,degree=1):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    
    #X_poly = (test_dataset - test_dataset.mean())/test_dataset.std()

    # X_poly = np.ones((test_dataset.shape[0], test_dataset.shape[1]))  # Add intercept term (column of ones)
    # for d in range(1, degree + 1):
    #     X_poly = np.hstack((X_poly, test_dataset ** d))
    y_pred = test_dataset.dot(w)
    #print(f"y_pred:{y_pred.shape}")

    # TODO

    return y_pred

### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to inclue the coefficients of your model in the report**



In [21]:
def calculate_mape(y_true, y_pred):
    # Convert inputs to numpy arrays to ensure they are arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate the absolute percentage error
    absolute_percentage_error = np.abs((y_true - y_pred) / y_true)
    
    
    # Compute the Mean Absolute Percentage Error
    mape = np.mean(absolute_percentage_error) * 100
    
    # print(f"y_true:{y_true}\n y_pred:{y_pred}\n absolute_percentage_error:{absolute_percentage_error}")
    return mape
def polynomial(data, degree):
    X_poly = np.ones((data.shape[0], 1)) 
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, data ** d)) 
    return X_poly
def gaussian(data,a,b):
    mu = np.mean(data[:,a:b])
    sigma = np.std(data[:,a:b])
    x = data[:,a:b]
    return np.exp(-1 * ((x - mu)**2) / (2*(sigma**2)))

def sigmoid_(data,a,b):
    data[:,a:b] = 1 / (1 + np.exp(-data[:,a:b]))
    return data[:,a:b]

def log_(data,a,b):
    data[:,a:b] = np.log(data[:,a:b] + 1)
    return data[:,a:b]

In [22]:
# # TODO

# # (1) Split data
# TraningData,ValidationData = SplitData(training_datalist,0.8)

# # (2) Preprocess data
# Training = PreprocessData(TraningData)
# Validation = PreprocessData(ValidationData)
# # (3) Train regression model

# w,loss,gradient,lossv,final_loss = Regression(Training,Validation)
# # (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
# y_predict = MakePrediction(w,ValidationData[:, :1])
# mape = calculate_mape(ValidationData[:, 1], y_predict)
# print(f"MAPE: {mape:.2f}%")
# # (5) Make prediction of testing dataset and store the values in output_datalist
# output_datalist = MakePrediction(w,testing_datalist)
# print(f"output_datalist : {output_datalist}")

### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [23]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [24]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [25]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [26]:

# TODO
# Change F to 0, M to 1
training_datalist[:, 1] = np.where(training_datalist[:, 1] == 'F', 0.0, 1.0)
training_datalist[:, -1] = np.where(training_datalist[:, -1] <= 0, np.nan, training_datalist[:, -1])
#training_datalist[:, 1].fillna(np.where(training_datalist[2] < 168, 0.6,1.0),inplace=True)

pre_test = testing_datalist[:,5:6] * testing_datalist[:,6:7] / testing_datalist[:,4:5] * testing_datalist[:,3:4]
testing_datalist[:, 1] = np.where(testing_datalist[:, 1] == 'F', 0.0, 1.0)
testing_datalist = np.column_stack((pre_test,testing_datalist))

# Change x to phi(x)
training_datalist = training_datalist.astype(float)
training_datalist[:,0:1] = gaussian(training_datalist,1,2)#training_datalist[:,1:2] = sigmoid_(training_datalist,4,5)
#training_datalist[:,2:3] = gaussian(training_datalist,1,2)
#training_datalist[:,3:4] = log_(training_datalist,4,5)
#training_datalist[:,4:5] = sigmoid_(training_datalist,1,2)
#training_datalist[:,5:6] = log_(training_datalist,4,5)
#training_datalist[:,6:7] = log_(training_datalist,1,2)

testing_datalist = testing_datalist.astype(float)
testing_datalist[:,1:2] = gaussian(testing_datalist,4,5)#testing_datalist[:,1:2] = sigmoid_(testing_datalist,1,2)
#testing_datalist[:,2:3] = gaussian(testing_datalist,4,5)
#testing_datalist[:,3:4] = log_(testing_datalist,1,2)
#testing_datalist[:,4:5] = sigmoid_(testing_datalist,4,5)
#testing_datalist[:,5:6] = log_(testing_datalist,1,2)
#testing_datalist[:,6:7] = log_(testing_datalist,4,5)


TraningData,ValidationData = SplitData(training_datalist,0.8)

# (2) Preprocess data
Training = PreprocessData(TraningData)
Validation = PreprocessData(ValidationData)

# Training[:,4:-1] = np.log(Training[:,4:5] + 1)
# Validation[:,4:-1] = np.log(Validation[:,4:5] + 1)
best_mape = float('inf')
best_randomseed = 6571
# for random_seed in range(9999):
#     r=random.randint(0,9999)
#     w,loss,gradient,lossv,final_loss = Regression(Training,Validation,degree=1 ,Rseed=r,num_iteration = 1000000,learning_rate = 0.000001)
#     # (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
#     y_predict = MakePrediction(w,Validation[:, : -1])
#     mape = calculate_mape(Validation[:, -1], y_predict)
#     # print(f"mape:{mape}, best:{best_mape}")
#     if mape < best_mape:
#         best_randomseed = r
#         best_mape = mape
#     if random_seed%10==0:
#         print(f"RandomSeed : {best_randomseed} , MAPE: {best_mape:.2f}% , loss:{final_loss}")

w,loss,gradient,lossv,final_loss = Regression(Training,Validation,degree=1 ,Rseed=best_randomseed,num_iteration = 1000000000,learning_rate = 0.0000000001)
#print(f"w:{w.shape} : Training : {Training[0]} : Test : {testing_datalist[0]}")

y_predict = MakePrediction(w,Validation[:, : -1])
mape = calculate_mape(Validation[:, -1], y_predict)
print(f"Best-RandomSeed : {best_randomseed} , MAPE: {mape:.2f}%")
# (5) Make prediction of testing dataset and store the values in output_datalist
output_datalist = MakePrediction(w,testing_datalist,degree=1)
print(f"Output = {output_datalist}")

# y_predict = MakePrediction(w,Training[:, : -1])
# mape = calculate_mape(Training[:, -1], y_predict)
# print(f"Training MAPE: {mape:.2f}%")

Iteration 0, Cost: 253331507.3136912, w:[0.91748328 0.44929869 0.56775438 0.97307774 0.94131692 0.56807708
 0.02780417 0.89824598 0.68727321], loss:484744309.35989773, gradient:1632675971.5097437
Iteration 5000, Cost: 6894.558892481178, w:[ 0.91743344 -0.00799989  0.56773428  0.97308516  0.93209742  0.56478979
  0.02589958  0.89403126  0.68030993], loss:13182.57433649575, gradient:20076.69788973896


Iteration 10000, Cost: 6794.396347213773, w:[ 0.9173955  -0.00793879  0.56772211  0.97310152  0.92490665  0.56232791
  0.02425195  0.89077471  0.67493068], loss:12991.207553580234, gradient:19929.457037501834
Iteration 15000, Cost: 6695.697511091631, w:[ 0.91735791 -0.00787813  0.56771008  0.97311784  0.91776884  0.55988433
  0.0226163   0.88754176  0.66959068], loss:12802.636764056564, gradient:19783.296135324443
Iteration 20000, Cost: 6598.440992969294, w:[ 0.91732064 -0.00781792  0.56769817  0.97313411  0.91068358  0.55745892
  0.02099254  0.88433225  0.66428965], loss:12616.821109616578, gradient:19638.20726292022
Iteration 25000, Cost: 6502.605714323009, w:[ 0.91728372 -0.00775815  0.56768639  0.97315034  0.9036505   0.55505154
  0.01938059  0.88114599  0.65902729], loss:12433.720329056188, gradient:19494.182558094537
Iteration 30000, Cost: 6408.170904681848, w:[ 0.91724712 -0.00769882  0.56767474  0.97316652  0.89666922  0.55266205
  0.01778037  0.87798282  0.65380334], loss:1225

In [27]:
def calculate_r_squared(X,y):
    X = np.column_stack((np.ones(X.shape[0]),X))
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

    y_pred = X @ beta_hat

    tss = np.sum((y-np.mean(y))**2)
    rss = np.sum((y-y_pred)**2)

    r_squared = 1 - (rss/tss)

    return r_squared

def calculate_vif(data):
    vif_values=[]
    if isinstance(data, np.ndarray):
        vif_data = pd.DataFrame(data)
    else:
        vif_data = data.copy()
    for i in range(vif_data.shape[1]):
        
        X = vif_data.drop(vif_data.columns[i],axis=1).values
        y = vif_data.iloc[:,i].values

        r_squared = calculate_r_squared(X,y)

        vif = 1 / (1-r_squared)
        vif_values.append(vif)

    return pd.Series(vif_values,index=vif_data.columns)

# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)

In [28]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# Graph

In [29]:
gradient = np.array(gradient)
plt.figure(figsize=(10, 6))
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))  # 1 row, 2 columns

g_features = ["D_S_BF_W","noise","age","gender","height","weight","bodyFat","diastolic","systolic"]
plt.plot(gradient, label=g_features)
plt.xlabel('Epoch')
plt.ylabel('Gradient')
plt.title('Gradient Curves')
plt.legend()  # Show legend
plt.grid(True)  # Show grid
plt.ylim(-100,100)
plt.xlim(0,20000)
# ax1.plot(gradient[:,0:7], label='Gradient', color='red')
# ax1.set_xlabel('Epoch')
# ax1.set_ylabel('Gradient')
# ax1.set_title('Gradient Curve')
# ax1.legend()
# ax1.grid(True)
# ax1.set_ylim(-50,50)
#ax1.set_xlim(0,50)
# Show the plots

# ax2.plot(gradient[:,7:], label='Gradient', color='red')
# ax2.set_xlabel('Epoch')
# ax2.set_ylabel('Gradient')
# ax2.set_title('Gradient Curve')
# ax2.legend()
# ax2.grid(True)
# ax2.set_ylim(-50,50)
# ax2.set_xlim(0,50)

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))  # Set the figure size

# Plot the first loss curve
plt.plot(loss, label='Loss', color='blue')

# Plot the second loss curve
plt.plot(lossv, label='Loss-V', color='red')

# Add labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()  # Show legend
plt.grid(True)  # Show grid
plt.ylim(0,500)
plt.xlim(0,20000)

# Show the plot
plt.show()

: 

In [ ]:
print(f"training : {Training.shape}")

training : (7032, 8)


In [ ]:
X_ = Training[:,5:6] * Training[:,6:7] / Training[:,4:5] * Training[:,3:4]
print(f"X_:{X_.shape}")
print(f"X_:{X_[0]}")
X_ = np.column_stack((X_,Training[:,:-1]))
print(f"X_:{X_.shape}")
print(f"X_:{X_[0]}")

X_:(7032, 1)
X_:[19183.24528302]
X_:(7032, 8)
X_:[1.91832453e+04 4.26525892e-01 0.00000000e+00 1.58000000e+02
 5.23000000e+01 2.65000000e+01 8.10000000e+01 1.20000000e+02]
